In [48]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 25
batch_size = 100
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified


In [50]:
## Residual block
## 1 3*3 conv
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    ## downsample => func to downsample residual input
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    

In [51]:
## resNet
class ResNet(nn.Module):
    def __init__(self, block, layers_nums, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.layer_1 = self.make_layer(block, 16, layers_nums[0])
        self.layer_2 = self.make_layer(block, 32, layers_nums[1], 2)
        self.layer_3 = self.make_layer(block, 64, layers_nums[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks_num, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels, stride=stride),
                                       nn.BatchNorm2d(out_channels))
        layers = []
        ## Only first block need downsample
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks_num):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
        
    def forward(self, x):
        ## before nesblock
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer_1(out)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [52]:
model = ResNet(ResidualBlock, [2,2,2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
# train
decay = 0
model.train()
for epoch in range(num_epochs):
    
    # Decay the learning rate every 20 epochs
    if (epoch+1) % 20 == 0:
        decay+=1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5**decay)
        print("The new learning rate is {}".format(optimizer.param_groups[0]['lr']))
        
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

Epoch [1/25], Step [100/500] Loss: 1.6824
Epoch [1/25], Step [200/500] Loss: 1.4397
Epoch [1/25], Step [300/500] Loss: 1.3942
Epoch [1/25], Step [400/500] Loss: 1.2124
Epoch [1/25], Step [500/500] Loss: 1.0865
Epoch [2/25], Step [100/500] Loss: 0.9777
Epoch [2/25], Step [200/500] Loss: 1.0034
Epoch [2/25], Step [300/500] Loss: 0.9149
Epoch [2/25], Step [400/500] Loss: 1.0463
Epoch [2/25], Step [500/500] Loss: 0.9041
Epoch [3/25], Step [100/500] Loss: 0.9735
Epoch [3/25], Step [200/500] Loss: 0.8835
Epoch [3/25], Step [300/500] Loss: 0.9457
Epoch [3/25], Step [400/500] Loss: 0.6469
Epoch [3/25], Step [500/500] Loss: 0.6864
Epoch [4/25], Step [100/500] Loss: 0.8057
Epoch [4/25], Step [200/500] Loss: 0.7061
Epoch [4/25], Step [300/500] Loss: 0.6749
Epoch [4/25], Step [400/500] Loss: 0.8572
Epoch [4/25], Step [500/500] Loss: 0.6676
Epoch [5/25], Step [100/500] Loss: 0.6918
Epoch [5/25], Step [200/500] Loss: 0.6068
Epoch [5/25], Step [300/500] Loss: 0.6933
Epoch [5/25], Step [400/500] Loss:

In [54]:
#Test
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 86.01 %
